In [ ]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from datetime import datetime
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict
from transformers import BertTokenizer,BertModel,BertTokenizerFast,DebertaTokenizer,DebertaTokenizerFast, AutoTokenizer
from utils.MIND import MIND
from utils.Manager import Manager

logger = logging.getLogger(__name__)

In [ ]:
# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = DebertaTokenizerFast.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")

In [ ]:
path = config.path + 'MIND/MINDlarge_test/behaviors.tsv'
behaviors = defaultdict(list)

with open(path, "r", encoding="utf-8") as rd:
    for idx in rd:
        impr_index, uid, time, history, impr = idx.strip("\n").split("\t")
        # important to subtract 1 because all list related to behaviors start from 0

        behaviors[uid].append([impr_index, uid, time, history, impr])

for k,v in behaviors.items():
    behaviors[k] = sorted(v,key=lambda x: datetime.strptime(x[2], "%m/%d/%Y %X %p"))

behavs = []
for k,v in behaviors.items():
    record = []
    imprs = []
    record.extend(v[0][:4])
    for i,behav in enumerate(v):
        imprs.append(behav[4])
    record.append(" ".join(imprs))
    behavs.append(record)

with open(path,"w",encoding="utf-8") as f:
    for record in behavs:
        f.write('\t'.join(record) + '\n')

In [ ]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'

# config.signal_length = 10
# config.scale = 'large'
# config.impr_size = 100
# config.mode = 'test'

# config.bert = 'microsoft/deberta-base'
# config.embedding = 'deberta'

# config.bert = 'bert-base-uncased
# config.embedding = 'bert'

manager = Manager(config)
path = manager.path + 'MIND/MINDdemo_train/'
a = MIND(manager, path + 'news.tsv', path + 'behaviors.tsv')

In [ ]:
subwords = pickle.load(open('data/cache/deberta/MINDdemo_train/news.pkl', 'rb'))

In [ ]:
news = t.convert_ids_to_tokens(a[0]['his_encoded_index'][1])
mask = a[0]['his_attn_mask'][1]
word = manager.convert_tokens_to_words(news)
subword = a[0]['his_subword_index'][1]
dedup = a[0]['his_refined_mask'][1]

for i,j,k,z in zip(news,mask,subword,dedup):
    print(i,j,k,z)

In [ ]:
x = a[1]
x['user_index']

In [ ]:
cdd = x['cdd_encoded_index'][0]

In [ ]:
subword = x['cdd_subword_index'][0]

In [ ]:
manager.convert_tokens_to_words(t.convert_ids_to_tokens(cdd))

In [ ]:
subword

In [ ]:
t.convert_ids_to_tokens(cdd)

In [ ]:
t.convert_ids_to_tokens([   1, 7565,   16, 4690,    0,    0,    0,    0])

In [ ]:
tokens = t.tokenize("[CLS] I love you embeddings.")
tokens

In [ ]:
news = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/deberta/MINDdemo_train/news.pkl', 'rb'))
ids = news['encoded_news']
subwords = news['subwords_all']

In [ ]:
t.convert_tokens_to_ids([i for i in r"[.&*()+=/\<>,!?;:~`@#$%^]"])